# 🎨 NeMo Data Designer: Synthetic Conversational Data with Person Details

> ⚠️ **Warning**: NeMo Data Designer is currently in Early Release and is not recommended for production use.

### 📚 What you'll learn

- This notebook demonstrates how to use the NeMo Data Designer to build a synthetic data generation pipeline step-by-step.

- We will create multi-turn user-assistant dialogues tailored for fine-tuning language models, enhanced with realistic person details. 

- These datasets could be used for developing and enhancing conversational AI applications, including customer \
support chatbots, virtual assistants, and interactive learning systems.


<br>

> 👋 **IMPORTANT** – Environment Setup
>
> - If you haven't already, follow the instructions in the [README](../../../README.md) to install the necessary dependencies.
>
> - You may need to restart your notebook's kernel after setting up the environment.
> - In this notebook, we assume you have a self-hosted instance of Data Designer up and running.
>
> - For deployment instructions, see the [Installation Options](https://docs.nvidia.com/nemo/microservices/latest/design-synthetic-data-from-scratch-or-seeds/index.html#installation-options) section of the [NeMo Data Designer documentation](https://docs.nvidia.com/nemo/microservices/latest/design-synthetic-data-from-scratch-or-seeds/index.html).


### 📦 Import the essentials

- The `data_designer` module of `nemo_microservices` exposes Data Designer's high-level SDK.

- The `essentials` module provides quick access to the most commonly used objects.


In [ ]:
from nemo_microservices.data_designer.essentials import (
    CategorySamplerParams,
    DataDesignerConfigBuilder,
    InferenceParameters,
    LLMJudgeColumnConfig,
    LLMStructuredColumnConfig,
    LLMTextColumnConfig,
    ModelConfig,
    NeMoDataDesignerClient,
    SamplerColumnConfig,
    SamplerType,
    Score,
    SubcategorySamplerParams
)

### ⚙️ Initialize the NeMo Data Designer Client

- `NeMoDataDesignerClient` is responsible for submitting generation requests to the microservice.


In [ ]:
NEMO_MICROSERVICES_BASE_URL = "http://localhost:8080"

data_designer_client = NeMoDataDesignerClient(base_url=NEMO_MICROSERVICES_BASE_URL)

### 🎛️ Define model configurations

- Each `ModelConfig` defines a model that can be used during the generation process.

- The "model alias" is used to reference the model in the Data Designer config (as we will see below).

- The "model provider" is the external service that hosts the model (see [the model config docs](https://docs.nvidia.com/nemo/microservices/latest/design-synthetic-data-from-scratch-or-seeds/configure-models.html) for more details).

- By default, the microservice uses [build.nvidia.com](https://build.nvidia.com/models) as the model provider.


In [ ]:
# This name is set in the microservice deployment configuration.
MODEL_PROVIDER = "nvidiabuild"

# The model ID is from build.nvidia.com.
MODEL_ID = "nvidia/nvidia-nemotron-nano-9b-v2"

# We choose this alias to be descriptive for our use case.
MODEL_ALIAS = "nemotron-nano-v2"

# This sets reasoning to False for the nemotron-nano-v2 model.
SYSTEM_PROMPT = "/no_think"

model_configs = [
    ModelConfig(
        alias=MODEL_ALIAS,
        model=MODEL_ID,
        provider=MODEL_PROVIDER,
        inference_parameters=InferenceParameters(
            temperature=0.6,
            top_p=0.95,
            max_tokens=1024,
        ),
    )
]

### 🏗️ Initialize the Data Designer Config Builder

- The Data Designer config defines the dataset schema and generation process.

- The config builder provides an intuitive interface for building this configuration.

- The list of model configs is provided to the builder at initialization.


In [ ]:
config_builder = DataDesignerConfigBuilder(model_configs=model_configs)

### Define Pydantic Models for Structured Outputs

You can use Pydantic to define a structure for the messages that are produced by Data Designer

In [ ]:
from typing import Literal
from pydantic import BaseModel, Field

class Message(BaseModel):
    """A single message turn in the conversation."""
    role: Literal["user", "assistant"] = Field(..., description="Which role is writing the message.")
    content: str = Field(..., description="Message contents.")


class ChatConversation(BaseModel):
    """A chat conversation between a specific user and an AI assistant.
    * All conversations are initiated by the user role.
    * The assistant role always responds to the user message.
    * Turns alternate between user and assistant roles.
    * The last message is always from the assistant role.
    * Message content can be long or short.
    * All assistant messages are faithful responses and must be answered fully.
    """
    conversation: list[Message] = Field(..., description="List of all messages in the conversation.")


class UserToxicityScore(BaseModel):
    """Output format for user toxicity assessment.

    Toxicity Scores:
    None: No toxicity detected in user messages.
    Mild: Slightly rude or sarcastic but not hateful or harmful.
    Moderate: Some disrespectful or harassing language.
    Severe: Overt hate, harassment, or harmful content.
    """
    reasons: list[str] = Field(..., description="Reasoning for user toxicity score.")
    score: Literal["None", "Mild", "Moderate", "Severe"] = Field(..., description="Level of toxicity observed in the user role responses.")

## 🎲 Adding Sampler Columns

- Sampler columns offer non-LLM based generation of synthetic data.

- They are particularly useful for **steering the diversity** of the generated data, as we demonstrate below.

In [ ]:
# Add domain column with subcategories for topics
config_builder.add_column(
    SamplerColumnConfig(
        name="domain",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=["Tech Support", "Personal Finances", "Educational Guidance"]
        )
    )
)

# Add topic subcategory
config_builder.add_column(
    SamplerColumnConfig(
        name="topic",
        sampler_type=SamplerType.SUBCATEGORY,
        params=SubcategorySamplerParams(
            category="domain",
            values={
                "Tech Support": [
                    "Troubleshooting a Laptop",
                    "Setting Up a Home Wi-Fi Network",
                    "Installing Software Updates",
                ],
                "Personal Finances": [
                    "Budgeting Advice",
                    "Understanding Taxes",
                    "Investment Strategies",
                ],
                "Educational Guidance": [
                    "Choosing a College Major",
                    "Effective Studying Techniques",
                    "Learning a New Language",
                ],
            },
        )
    )
)

# Add complexity column
config_builder.add_column(
    SamplerColumnConfig(
        name="complexity",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=["Basic", "Intermediate", "Advanced"]
        )
    )
)

# Add conversation length column
config_builder.add_column(
    SamplerColumnConfig(
        name="conversation_length",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=[2, 4, 6, 8]
        )
    )
)

# Add user mood column
config_builder.add_column(
    SamplerColumnConfig(
        name="user_mood",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=["happy", "silly", "sarcastic", "combative", "disappointed", "toxic"]
        )
    )
)

## 🦜 Adding LLM Generated columns
Now define the columns that the model will generate. These prompts instruct the LLM to produce the actual conversation: 
- a system prompt to guide how the AI assistant engages in the conversation with the user, 
- the conversation, and 
- finally, we generate a toxicity_label to assess user toxicity over the entire conversation.
<br> 

### 💬🤖 AI Assistant system prompt and conversation

We generate a system prompt to base the AI assistant and then generate the entire conversation.

In [ ]:
# Generate assistant system prompt
config_builder.add_column(
    LLMTextColumnConfig(
        name="assistant_system_prompt",
        system_prompt=SYSTEM_PROMPT,
        prompt=("Write a reasonable system prompt for a helpful AI assistant with expertise in "
                "{{domain}} and {{topic}}. The AI assistant must not engage in harmful behaviors."),
        model_alias=MODEL_ALIAS,
    )
)

# Generate the user's task
config_builder.add_column(
    LLMTextColumnConfig(
        name="user_task",
        system_prompt=SYSTEM_PROMPT,
        prompt="Define a simple task related to {{topic}} of {{complexity}} complexity for the user.",
        model_alias=MODEL_ALIAS,
    )
)

# Generate the conversation
config_builder.add_column(
    LLMStructuredColumnConfig(
        name="conversation",
        system_prompt=SYSTEM_PROMPT,
        prompt=(
            "<task>\n{{user_task}}\n</task>\n\n"
            "<system_prompt>{{assistant_system_prompt}}</system_prompt>\n\n"
            "Generate a conversation between a user and an AI assistant with <system_prompt> about <task>.\n"
            "User is asking the assistant for advice and is in a {{user_mood}} mood.\n"
            "The conversation must be {{conversation_length}} messages in length.\n"
            "The conversation must come to a natural end in {{conversation_length}} messages and if the assistant is unable "
            "to solve the user's needs by then, they should offer to continue the conversation later or redirect them to additional resources.\n"
            "The conversation must be realistic and natural, incorporating details from both the user's and assistant's profiles.\n"
            "The AI assistant role has the system prompt defined in <system_prompt>. All of its responses must conform to <system_prompt>:\n"
        ),
        output_format=ChatConversation,
        model_alias=MODEL_ALIAS,
    )
)


## 🔍 LLM-as-a-Judge: Toxicity Assessment

When generating our synthetic dataset, we need to determine the quality of the generated dialogs. \
We use the LLM-as-a-Judge strategy to do this. 

To do so, we need to define the rubric that the LLM should use to assess generation quality along with a prompt 
that provides relavant instructions. 

In [ ]:
# Toxicity Label for the entire conversation
# Define Toxicity Rubric
toxicity_rubric = Score(
    name="Toxicity",
    description="Evaluation of harmful, harassing, or hateful language in conversations",
    options=
        {
            "4": "No toxicity detected; conversation is respectful and appropriate.",
            "3": "Mild toxicity; slightly rude or sarcastic but not hateful or harmful.",
            "2": "Moderate toxicity; contains disrespectful or harassing language.",
            "1": "Severe toxicity; contains overt hate, harassment, or harmful content.",
            "0": "Extreme toxicity; contains dangerous, threatening, or abusive content."
        }

)

TOXICITY_PROMPT_TEMPLATE = (
    "You are an expert in content moderation and digital communication ethics. "
    "You are evaluating the toxicity level in user messages.\n"
    "Take a deep breath and carefully analyze the conversation provided. \n"
    "Use the rubric to score the level of toxicity in the USER'S messages only.\n"

    "Consider:\n"
    "- Focus primarily on harassment, hate, or harmful language\n"
    "- Evaluate the full context of the conversation\n"
    "- Be objective and consistent in your assessment\n"
    "- If any single message exhibits toxicity, it should influence the final score\n"

    "## CONVERSATION\n"
    "{{ conversation }}"
)

config_builder.add_column(
    LLMJudgeColumnConfig(
        name="toxicity_evaluation",
        system_prompt=SYSTEM_PROMPT,
        prompt=TOXICITY_PROMPT_TEMPLATE,
        scores=[toxicity_rubric],
        model_alias=MODEL_ALIAS
    )
)

### 🔁 Iteration is key – preview the dataset!

1. Use the `preview` method to generate a sample of records quickly.

2. Inspect the results for quality and format issues.

3. Adjust column configurations, prompts, or parameters as needed.

4. Re-run the preview until satisfied.

In [ ]:
# Preview a few records
preview = data_designer_client.preview(config_builder)

In [ ]:
# More previews
preview.display_sample_record()

### 📊 Analyze the generated data

- Data Designer automatically generates a basic statistical analysis of the generated data.

- This analysis is available via the `analysis` property of generation result objects.


In [ ]:
# Print the analysis as a table.
preview.analysis.to_report()

### 🆙 Scale up!

- Happy with your preview data?

- Use the `create` method to submit larger Data Designer generation jobs.


In [ ]:
job_results = data_designer_client.create(config_builder, num_records=20)

# This will block until the job is complete.
job_results.wait_until_done()

In [ ]:
# Load the generated dataset as a pandas DataFrame.
dataset = job_results.load_dataset()

dataset.head()

In [ ]:
# Load the analysis results into memory.
analysis = job_results.load_analysis()

analysis.to_report()

In [ ]:
TUTORIAL_OUTPUT_PATH = "data-designer-tutorial-output"

# Download the job artifacts and save them to disk.
job_results.download_artifacts(
    output_path=TUTORIAL_OUTPUT_PATH,
    artifacts_folder_name="artifacts-community-contributions-multi-turn-chat",
);